# ICMAP-Elites DEMO

## Imports

In [1]:
from pcgsepy.common.jsonifier import json_dumps, json_loads
from pcgsepy.evo.fitness import (Fitness, bounding_box_fitness,
                                 box_filling_fitness, func_blocks_fitness,
                                 mame_fitness, mami_fitness)
from pcgsepy.evo.genops import expander
from pcgsepy.fi2pop.utils import MLPEstimator
from pcgsepy.mapelites.behaviors import (BehaviorCharacterization, avg_ma,
                                         mame, mami, symmetry)
from pcgsepy.mapelites.buffer import Buffer, max_merge, mean_merge, min_merge
from pcgsepy.mapelites.emitters import HumanPrefMatrixEmitter
from pcgsepy.mapelites.map import MAPElites
from pcgsepy.mapelites.webapp import app, set_app_layout, set_callback_props
from pcgsepy.setup_utils import get_default_lsystem, setup_matplotlib


## Settings & L-system initialization

In [2]:
load_existing_mapelites = True

In [3]:
setup_matplotlib(larger_fonts=False)

used_ll_blocks = [
    'MyObjectBuilder_CubeBlock_LargeBlockArmorCornerInv',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorCorner',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorBlock',
    'MyObjectBuilder_Gyro_LargeBlockGyro',
    'MyObjectBuilder_Reactor_LargeBlockSmallGenerator',
    'MyObjectBuilder_CargoContainer_LargeBlockSmallContainer',
    'MyObjectBuilder_Cockpit_OpenCockpitLarge',
    'MyObjectBuilder_Thrust_LargeBlockSmallThrust',
    'MyObjectBuilder_InteriorLight_SmallLight',
    'MyObjectBuilder_CubeBlock_Window1x1Slope',
    'MyObjectBuilder_CubeBlock_Window1x1Flat',
    'MyObjectBuilder_InteriorLight_LargeBlockLight_1corner'
]

lsystem = get_default_lsystem(used_ll_blocks=used_ll_blocks)

In [4]:
expander.initialize(rules=lsystem.hl_solver.parser.rules)

## CMAP-Elites

In [5]:
feasible_fitnesses = [
    #     Fitness(name='BoundingBox',
    #             f=bounding_box_fitness,
    #             bounds=(0, 1)),
    Fitness(name='BoxFilling',
            f=box_filling_fitness,
            bounds=(0, 1)),
    Fitness(name='FuncionalBlocks',
            f=func_blocks_fitness,
            bounds=(0, 1)),
    Fitness(name='MajorMediumProportions',
            f=mame_fitness,
            bounds=(0, 1)),
    Fitness(name='MajorMinimumProportions',
            f=mami_fitness,
            bounds=(0, 1))
]

behavior_descriptors = [
    BehaviorCharacterization(name='Major axis / Medium axis',
                             func=mame,
                             bounds=(0, 10)),
    BehaviorCharacterization(name='Major axis / Smallest axis',
                             func=mami,
                             bounds=(0, 20)),
    BehaviorCharacterization(name='Average Proportions',
                             func=avg_ma,
                             bounds=(0, 20)),
    BehaviorCharacterization(name='Symmetry',
                             func=symmetry,
                             bounds=(0, 1))
]

behavior_descriptors_names = [x.name for x in behavior_descriptors]

In [6]:
if load_existing_mapelites:
    with open('me.json', 'r') as f:
        mapelites = json_loads(f.read())
else:
    mapelites = MAPElites(lsystem=lsystem,
                          feasible_fitnesses=feasible_fitnesses,
                          behavior_descriptors=(behavior_descriptors[0], behavior_descriptors[1]),
                          estimator=MLPEstimator(xshape=len(feasible_fitnesses),
                                                 yshape=1),
                          buffer = Buffer(merge_method=mean_merge),
                          emitter=HumanPrefMatrixEmitter(),
                          n_bins=(8, 8))
    mapelites.generate_initial_populations()
    with open('me.json', 'w') as f:
        f.write(json_dumps(mapelites))



## Webapp

In [7]:
set_callback_props(mapelites=mapelites)

set_app_layout(mapelites=mapelites)

In [8]:
# app.run_server(debug=True, host='0.0.0.0', port = 8080, use_reloader=False)
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app 'pcgsepy.mapelites.webapp' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
